<a href="https://colab.research.google.com/github/esergiob/GBDT/blob/master/XGBoost_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast Machine Learning Algorithms

Gradient boosting decision tree (**GBDT**) is one of the best performing classes of algorithms in machine learning competitions. One implementation of the gradient boosting decision tree – xgboost – is one of the most popular algorithms on Kaggle. Among the 29 challenge winning solutions published at Kaggle’s blog during 2015, 17 used xgboost. If you take a look at the kernels in a Kaggle competition, you can clearly see how popular xgboost is.

Though xgboost seemed to be the go-to algorithm in Kaggle for a while, a new contender is quickly gaining traction: lightGBM. Released from Microsoft, this algorithm has been claimed to be more efficient (better predictive performance for the same running time) than xgboost.

One thing that can be confusing is the difference between xgboost, lightGBM and Gradient Boosting Decision Trees (which we will henceforth refer to as GBDTs). Xgboost and lightGBM are both subtypes/specific instances of the GBDT algorithm. Though they both implement the same underlying algorithm, they each introduce various tricks to make training more efficient or to improve performance. 

## The Basics of Boosted Decision Trees

Gradient boosting is a machine learning technique that produces a prediction model in the form of an ensemble of weak classifiers, optimizing a differentiable loss function. One of the most popular types of gradient boosting is boosted decision trees, that internally is made up of an ensemble of weak decision trees. There are two different strategies to compute the trees: level-wise and leaf-wise. The level-wise strategy grows the tree level by level. In this strategy, each node splits the data prioritizing the nodes closer to the tree root. The leaf-wise strategy grows the tree by splitting the data at the nodes with the highest loss change. Level-wise growth is usually better for smaller datasets whereas leaf-wise tends to overfit. Leaf-wise growth tends to excel in larger datasets where it is considerably faster than level-wise growth.

A key challenge in training boosted decision trees is the **computational cost of finding the best split for each leaf**. Conventional techniques find the **exact split** for each leaf, and require scanning through all the data in each iteration. A different approach **approximates the split** by building histograms of the features. That way, the algorithm doesn’t need to evaluate every single value of the features to compute the split, but only the bins of the histogram, which are bounded. This approach turns out to be much more efficient for large datasets, without adversely affecting accuracy.

**XGBoost started in 2014**, and it has become popular due to its use in many **winning Kaggle competition entries**. Originally XGBoost was based on a level-wise growth algorithm, but recently has added an option for leaf-wise growth that implements split approximation using histograms. We refer to this version as XGBoost hist. LightGBM is a more recent arrival, started in March 2016 and open-sourced in August 2016. It is based on a leaf-wise algorithm and histogram approximation, and has attracted a lot of attention due to its speed. Apart from multithreaded CPU implementations, GPU acceleration is now available on both XGBoost and LightGBM too.


## Implementation on a Dataset
I am using the Kaggle Dataset of flight delays for the year 2015 as it has both categorical and numerical features. With approximately 5 million rows, this dataset will be good for judging the performance in terms of both speed and accuracy of tuned models for each type of boosting. I will be using a 10% subset of this data ~ 500k rows.

In [0]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv("flights.csv")
data = data.sample(frac = 0.1, random_state=10)

data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT", "ORIGIN_AIRPORT",
             "AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]

FileNotFoundError: ignored

In [0]:
data.dropna(inplace=True)

In [0]:
data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1

cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), 
        data["ARRIVAL_DELAY"], random_state=10, test_size=0.25)

In [0]:
X_train.shape

In [0]:
X_train.head()

## XGBoost

In [0]:
import xgboost as xgb
from sklearn import metrics

In [0]:
# Receiver operating characteristic (ROC) 
# Based on the ROC curve, we can then compute the so-called ROC area under
# the curve (ROC auc) to characterize the performance of a classification model.
def auc(m, X_train, X_test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(X_train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(X_test)[:,1]))

In [0]:
# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}

from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3,                                  verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

In [0]:
model = xgb.XGBClassifier(max_depth=50, min_child_weight=1, n_estimators=200, n_jobs=-1 , verbose=1,learning_rate=0.16)

model.fit(X_train,y_train)

auc(model, X_train, X_test)

## LightGBM

In [0]:
import lightgbm as lgb

def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_test,m.predict(test)))

lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_

d_train = lgb.Dataset(X_train, label=y_train)
params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 300}

In [0]:
# Without Categorical Features
model2 = lgb.train(params, d_train)
auc2(model2, X_train, X_test)

# With Catgeorical Features
cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT"]
model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
auc2(model2, X_train, X_test)